Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [29]:
import sys
import os
# Get the path to the project root directory (PBGSK)
# 'os.path.abspath('..')' goes one level up from the 'notebooks' directory
project_root = os.path.abspath('..')
print(f"Project root directory: {project_root}")
# Add the project root to the system path if it's not already there
if project_root not in sys.path:
    sys.path.append(project_root)

from src.utils.data_importer import data_loader


Project root directory: c:\Users\romul\Repositories\pBGSK


In [13]:
import pandas as pd
import random
import numpy as np
import sys
import sklearn as sk

In [ ]:
df = data_loader('tic_tac_toe_endgame')


In [19]:
df.data.features

,top-left-square,top-middle-square,top-right-square,middle-left-square,middle-middle-square,middle-right-square,bottom-left-square,bottom-middle-square,bottom-right-square
0,x,x,x,x,o,o,x,o,o
1,x,x,x,x,o,o,o,x,o
2,x,x,x,x,o,o,o,o,x
3,x,x,x,x,o,o,o,b,b
4,x,x,x,x,o,o,b,o,b
...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x
954,o,x,o,x,x,o,x,o,x
955,o,x,o,x,o,x,x,o,x
956,o,x,o,o,x,x,x,o,x


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
#from ucimlrepo import fetch_dataset

In [42]:

class Individual:
    def __init__(self,data_set_name: str,individual_id :int, features: list,X_train,X_test,y_train, y_test):
        """Cada individuo possui um vetor binário representando a seleção de features."""
        self.data_set_name = data_set_name
        self.number_of_features = int(np.sum(features))
        self.individual_id = individual_id
        self.features = features
        
        self.X_train = X_train
        self.X_test= X_test
        self.y_train = y_train
        self.y_test = y_test
        #sk.model_selection.train_test_split(self.features, self.targets, test_size=0.33, random_state=42)
        self.score = 0.0
        self.classifier = KNeighborsClassifier(n_neighbors=5)
        
    def calculate_fitness(self):
        """ acuracia"""
    # Get the indices of the selected features (where feature == 1)
        selected_indices = np.where(self.features == 1)[0]
        
        if len(selected_indices) == 0:
            self.score = 0.0
            return

        # Select the features from the data
        X_train_selected = self.X_train[:, selected_indices]
        X_test_selected = self.X_test[:, selected_indices]

        # Train and score the model
        self.classifier.fit(X_train_selected, self.y_train)
        y_pred = self.classifier.predict(X_test_selected)
        self.score = accuracy_score(self.y_test, y_pred)
class Population:
    def __init__(self, individuals: list):
        self.individuals = individuals
        

In [49]:
def population_creation(num_population:int, lower_k :int, upper_k:int,data_tupla,data_set_name) -> Population:
    """Criando uma população inicial de indivíduos. com features variando entre lower_k e upper_k.
    num_population: número de indivíduos na população"""
    X_train, X_test, y_train, y_test = data_tupla
    population = []
    total_features = X_train.shape[1]
    for i in range(num_population):
        k = lower_k + random.random()*(upper_k-lower_k)
        k = int(k)
        features = random.sample(range(0, total_features), k)
        bin_feature = np.zeros(total_features)

        for feature in features:
            bin_feature[feature] = 1  # Mark selected

        indiv = Individual(data_set_name=data_set_name,
                individual_id=i,
                features=bin_feature,
                X_train=X_train,  # Pass the reference
                y_train=y_train,  # Pass the reference
                X_test=X_test,    # Pass the reference
                y_test=y_test     # Pass the reference
            )
        population.append(indiv)
    pop =Population(population)
    return pop

In [50]:
def stage_dimension(gem_max:int, gem_atual:int,k_rate:float, num_individuos:int):
    """gem_max é numero máximo de gerações
        gem_atual é a geração atual
        d_junior é o número de indivíduos juniores
        d_senior é o número de indivíduos seniores
        k_rate é o knowledge rate"""
        
    d_junior = int(num_individuos*((gem_max - gem_atual)/gem_max)**k_rate)
    d_senior = num_individuos - d_junior
    return d_junior, d_senior


In [51]:
#pop =population_creation(num_population =15,data_set_name = 'tic', lower_k=3, upper_k=6, total_features=9)

    

In [52]:
from sklearn.metrics import accuracy_score
import numpy as np
data_set_name = 'breast_cancer'
data= data_loader(data_set_name)
X =df.data.features
y = df.data.targets
data_tuple = sk.model_selection.train_test_split(X, y, test_size=0.33, random_state=42)
# 2. Create the population
# This passes the data references to all individuals
pop = population_creation(
    num_population=50,
    lower_k=3,
    upper_k=10,
    data_tupla=data_tuple,
    data_set_name=data_set_name
)

# 3. Calculate fitness for each individual
for indiv in pop.individuals:
    indiv.calculate_fitness()
    print(f"Individual {indiv.individual_id} Features: {indiv.number_of_features} Score: {indiv.score:.4f}")

InvalidIndexError: (slice(None, None, None), array([0, 1, 2, 3, 4, 5, 8]))